# Intuitive Neural Networks by Hand

This notebook shows how to build a neural network by hand.  The motivation for creating _yet another_ neural network from scratch post is that I believe most get lost in the calculus and lose the concepts.  Additionally, I believe the demonstation code is often designed to be efficient with numpy, not illustrative.

Below, we create classes to represent our neural network and then use it to fit to a simple example from the Iris dataset.

* NOTE: Code is still under development and bugs likely exist.

# Concepts

Neural networks are essentially two things:
1. A collection of node layers with connecting weights
1. A forwards-backwards algorithm (of sorts) for optimizing those weights.

The forward-backward algorithm is a common paradigm outside of neural networks and is helpful to learn about. By analogy, the forward step simply computes the current outcome probabilities.  The backwards step updates the weights used in making predictions to those that seem best at the current iteration.  This is the "gradient descent" that is frequently mentioned (and used).  The bidirectional process continues (hopefully) to convergence.

At prediction time, one is essentially re-computing the forward step of the neural network.

<img src="static/neural-network.png" alt="drawing" width="500"/>

A key concept is that neural network weights are linear, with each node receiving a linear combination of the node outputs from the prior layer. For the first layer, nodes receive a linear combination of the dataset features.  For subsequent layers, nodes receive a linear combination of the activation functions of the previous nodes.  In this way, neural networks continually augment the data with linear combinations (followed by non-linear activation functions), projecting iterations of data into new dimensions until a good prediction is made.  As layers get arbitrarily large, one can imagine how these projections capture latent features.

Now notice that the weights between these layers are simply edges on a bipartite graph.  This gives us our insight that we can represent the weights as a matrix between the current layer and the prior layer:

<img src="static/bipartite-graph.png" alt="drawing" width="500"/>

Thus it follows that weighting the input is simply a matrix multiplication by the adjacency matrix: _W*input + b_.

## Model Layer (Incomplete Version)

The first thing that our neural network will need is a notion of a layer.  As with any coding project, we do not know our final state API *yet*, but we have a notion of how to start.  Namely, our Layer will need a size (number of nodes), an activation function, weights, and betas.  We also use a locator pattern to store to which model iteration this Layer belongs (note that we have not defined model iterations yet!).

The IncompleteLayer class knows how to apply weights using our bipartite graph insight.  We simply multiply the weights by the input matrix and add our beta terms.

In [ ]:
import numpy as np

class IncompleteLayer:
    def __init__(self, model_iteration, size, activation_function, weights, betas):
        self.model_iteration = model_iteration  # locator pattern
        self.size = size
        self.activation_function = activation_function
        self.weights = weights
        self.betas = betas

    def apply_weights(self, layer_input):
        """Equivalent to a forward propegation on a single layer"""
        Z = np.dot(self.weights, layer_input) + self.betas
        output = self.activation_function(Z)
        return output

## Model Iteration

Every neural network model runs for multiple iterations and these iterations have their own layers, weights, etc.  Thus it makes for a good class to build.

Our ModelIteration class should have a `feed_forward` function that runs our `Layer.apply_weights` and then returns the last value (the output of the last layer is the neural network output).  Similarly, predict is simply a call to `feed_forward` under this methodology.

We will also need a way to do backprogegation.  The derivative calculations can be passed to Layers.  (Note that we have implicitly defined `Layer.update_weights`.)

### Setting Weights

Despite defining much of the model iterations and layers, we never wrote any code that sets the values of weights. 

**On any forward feeding of a model, the weights are defined to be either random initialization or the weights from the prior backpropegation.** Thus ModelIteration will need to loop to create the layers, each time looking at the prior iteration values:

```
for layer in model_structure:
    if prior_iteration:
        weights = prior_iteration.weights
        betas = prior_iteration.betas
    else:
        weights = random_initialization()
        betas = zeros()
```

A final wrinkle: To initialize the weights on the first iteration, we must be able to define the dimensions of the weight matrix.  From our bipartite graph, the matrix dimensions will be *(# current layer nodes, # prior layer nodes)*.  However, remember that the first layer has *# dataset features* columns as there is no prior layer.  The `__init__` method of the ModelIteration class updates the pseudocode to include these dimensions.

In [ ]:
import numpy as np

class ModelIteration:
    def __init__(self, model, data, Y, learning_rate, prior_iteration=None):
        self.model = model  # locator pattern
        self.learning_rate = learning_rate
        self.prior_iteration = prior_iteration
        self.layers = []
        
        # iterate over each layer to initialize weights, per the above description.
        for layer_number, layer in enumerate(self.model.model_structure):
            if self.prior_iteration is None: # first iteration, must initialize weights
                if 0 == layer_number:
                    prior_layer_size = data.shape[1]
                else:
                    prior_layer_size = self.layers[-1].size
                weights = np.random.randn(layer["size"], prior_layer_size)
                betas = np.zeros((layer["size"], 1))
            else:
                weights = self.prior_iteration.layers[layer_number].weights # backprop output
                betas = self.prior_iteration.layers[layer_number].betas            
           
            layer = Layer(self, layer["size"], layer["activation"], weights, betas)
            self.layers.append(layer)
        
    def feed_forward(self, data):
        prior_output = self.model.data.T
        for layer in self.layers:
            output = layer.apply_weights(prior_output)
            prior_output = output
        return output
    
    def predict(self, data):
         return self.feed_forward(data)
        
    def evaluate(self, data=None, Y=None):
        if data is None:
            data = self.model.data
            Y = self.model.Y
        predictions = self.predict(data)
        return self.model.cost_function(predictions, Y)

    def propegate_backward(self):
        derivatives_list = []
        for i, layer in enumerate(self.layers):
            derivatives = layer.calculate_derivatives()
            derivatives_list.append(derivatives)
            
        for i, layer in enumerate(self.layers):
            layer.update_derivatives(derivatives_list[i])
            layer.update_weights(self.learning_rate)

## Model Class

Our model class is effectively a wrapper for ModelIteration that calls the last iteration for `predict`.  The only difference is that our Model class needs a `train` method.  `train` will loop over model iterations, feed forward and back propegate, printing status along the way.

In [ ]:
import numpy as np

class Model:
    def __init__(self, data, Y, model_structure, cost_function, learning_rate):
        self.data = data
        self.Y = Y
        self.model_structure = model_structure
        self.cost_function = cost_function
        self.learning_rate = learning_rate
        self.iterations = None
        
    def train(self, learning_rate=0.01, num_iterations=5000):
        self.iterations = []
        prior_iteration = None
        for iteration in range(num_iterations):
            model_iteration = ModelIteration(self, self.data, self.Y, learning_rate, prior_iteration)
            self.iterations.append(model_iteration)
            
            iteration_output = model_iteration.feed_forward(self.data)
            model_iteration.propegate_backward() # update weights
    
            prior_iteration = model_iteration
            
            if iteration % 5 == 0:
                print("Completed iteration {}.  Loss: {}".format(iteration, self.evaluate(self.data, self.Y)))
                
        return self.evaluate(self.data, self.Y)
            
    def predict(self, data=None):
        self.assert_trained()
        if data is None:
            data = self.data
        return self.iterations[-1].predict(data)
    
    def evaluate(self, data=None, Y=None):
        self.assert_trained()
        if data is None:
            data = self.data
            Y = self.Y
        return self.iterations[-1].evaluate(data, Y)
    
    def assert_trained(self):
        if self.iterations is None:
            raise Exception("Must train before running `predict`.")


## Layer Version (Complete Version)

Now we need to update Layer for backpropegation.  We can start by making an `update_weights` function that executes the core premise: 

    new_weights = old_weights - learning_rate * derivatives

where *derivatives* is the derivative of the loss function with respect to the weights.  This is the step where neural network introductions appear to get complex because of the differential calculus.  However, we can always make a method to approximate the derivative.  Thus, we can (in theory) approximate a derivative for any loss function we seek to minimize.

Better estimators are more involved so we'll simply look at the slope for the line between the points *W-e* and *W+e*, where *W* are the weights and *e* is some small number epsilon.  

In [ ]:
import numpy as np

class Layer:
    def __init__(self, model_iteration, size, activation_function, weights, betas):
        self.model_iteration = model_iteration  # locator pattern
        self.size = size
        self.activation_function = activation_function
        self.weights = weights
        self.betas = betas
        self.derivatives = None

    def apply_weights(self, layer_input):
        Z = np.dot(self.weights, layer_input) + self.betas
        output = self.activation_function(Z)
        return output
    
    def update_weights(self, learning_rate):
        if self.derivatives is None:
            self.calculate_derivatives()
        self.weights = self.weights - learning_rate * self.derivatives

    def update_derivatives(self, derivatives):
        self.derivatives = derivatives
        
    def calculate_derivatives(self):
        derivatives = []
        offset = DERIVATIVE_OFFSET
        
        for i in range(self.weights.shape[0]):
            for j in range(self.weights.shape[1]):
                original_weight = self.weights[i][j]
                
                self.weights[i][j] = original_weight - offset
                cost1 = self.model_iteration.evaluate()
                
                self.weights[i][j] = original_weight + offset
                cost2 = self.model_iteration.evaluate()
                
                derivative = (cost2 - cost1) / (2 * DERIVATIVE_OFFSET)
                self.weights[i][j] = original_weight
                derivatives.append(derivative)
        return np.array(derivatives).reshape(*self.weights.shape)

## Activation Functions

Our activation functions are generic.

In [ ]:
import numpy as np

def relu_activation(x):
    """Vectorized relu activation function
    :return: 0 if x is less than 0, x otherwise.
    """
    return np.multiply(x, x >= 0)
    
def sigmoid_activation(x):
    """Vectorized sigmoid activation function
    :return: sigmoid of x
    """
    return 1. / (1 + np.exp(-x))


## Loss Functions

Because we used numerical methods to estimate the derivatives of the loss function, we aren't required to use a function whose derivative we know.  However, we'll use the same log loss regardless as it is appropriate to the domain.

In [ ]:
from sklearn.metrics import log_loss

def binary_loss_function(predictions, Y):
    """Loss function for a binary classifier"""
    return log_loss(Y, (predictions[0] > 0.5).astype(int))


## Putting It Together

In [ ]:
from sklearn import datasets
from sklearn import preprocessing

DERIVATIVE_OFFSET = 0.1

iris = datasets.load_iris()
iris_x = preprocessing.scale(iris["data"])
iris_y = iris["target"]
Y2 = (iris_y == 0).astype(int)

structure = [{"size": 3, "activation": relu_activation}, 
             {"size": 1, "activation": sigmoid_activation}]

model = Model(iris_x, Y2, structure, binary_loss_function, learning_rate=0.001)
model.train()



  
        
    


       
        
